# 📊 Social Interaction Analysis

Analyze indoor tracking data for social interactions.

In [1]:
# =============================================================================
# IMPORTS & SETUP
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# Global state
state = {
    'raw_data': None,
    'processed_data': None,
    'results': None,
    'metrics': None,
    'tags_dict': None,
    'dates_list': None
}

print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


In [2]:
# =============================================================================
# PROCESSING FUNCTIONS
# =============================================================================

def apply_exclusions(df, excl):
    if not excl: return df
    df = df.copy()
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df['_d'] = df['TimeStamp'].dt.strftime('%d/%m/%Y')
    df['_t'] = df['TimeStamp'].dt.strftime('%H:%M:%S')
    mask = pd.Series(False, index=df.index)
    for date, periods in excl.items():
        for s, e in periods:
            s = s+':00' if len(s)==5 else s
            e = e+':00' if len(e)==5 else e
            m = (df['_d']==date) & (df['_t']>=s) & (df['_t']<=e)
            mask = mask | m
    return df.drop(columns=['_d','_t'])[~mask]

def resample_data(dfl, ts, progress_callback=None):
    res = []
    for df_i in dfl:
        df_i = df_i.copy()
        df_i['TimeStamp'] = pd.to_datetime(df_i['TimeStamp'])
        resampled = []
        tags = df_i['TagId'].unique()
        for idx, t in enumerate(tags):
            if progress_callback:
                progress_callback(idx / len(tags) * 0.25, f'Resampling tag {idx+1}/{len(tags)}')
            g = df_i[df_i['TagId']==t].set_index('TimeStamp').sort_index()
            gn = g.select_dtypes(include='number').resample(f'{ts}ms').mean()
            gn = gn.interpolate('linear', limit_direction='both')
            gn['TagId'] = t
            resampled.append(gn.reset_index())
        res.append(pd.concat(resampled, ignore_index=True))
    return res

def reindex_data(dfl, ts):
    df = pd.concat(dfl, ignore_index=True) if isinstance(dfl,list) else dfl
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df['Date'] = df['TimeStamp'].dt.strftime('%d/%m/%Y')
    df['Time'] = pd.to_datetime(df['TimeStamp'].dt.strftime('%H:%M:%S.%f'))
    tags = dict(enumerate(df['TagId'].unique()))
    dates = df['Date'].unique()
    df = df.drop(columns=['TimeStamp','Z'], errors='ignore').set_index(['Date','TagId'])
    return df, tags, dates

def fill_smooth(df, ts, sw, se, progress_callback=None):
    t0 = df.groupby(level=['Date','TagId'])['Time'].first().min()
    t1 = df.groupby(level=['Date','TagId'])['Time'].last().max()
    rng = pd.date_range(t0, t1, freq=f'{ts}ms')
    res = []
    groups = list(df.groupby(['Date','TagId']))
    for idx, ((d,t), g) in enumerate(groups):
        if progress_callback:
            progress_callback(0.25 + idx / len(groups) * 0.25, f'Smoothing {idx+1}/{len(groups)}')
        gf = g.set_index('Time').reindex(rng)
        gf['Time'] = rng
        gf['X'] = gf['X'].rolling(sw, min_periods=1, center=True).mean()
        gf['Y'] = gf['Y'].rolling(sw, min_periods=1, center=True).mean()
        gf['X Error'], gf['Y Error'] = se, se
        gf['Date'], gf['TagId'] = d, t
        res.append(gf.reset_index(drop=True))
    return pd.concat(res, ignore_index=True).set_index(['Date','TagId'])

def calc_vel(df, ts, progress_callback=None):
    dt = ts/1000
    res = []
    groups = list(df.groupby(['Date','TagId']))
    for idx, ((d,t), g) in enumerate(groups):
        if progress_callback:
            progress_callback(0.5 + idx / len(groups) * 0.1, f'Velocity {idx+1}/{len(groups)}')
        x, y = g['X'].values.astype(float), g['Y'].values.astype(float)
        dx, dy = np.diff(x, prepend=np.nan), np.diff(y, prepend=np.nan)
        v = np.sqrt(dx**2+dy**2)/dt
        sub = pd.DataFrame({'Time':g['Time'].values, 'Velocity':v.astype(float)})
        sub['Date'], sub['TagId'] = d, t
        res.append(sub)
    return pd.concat(res, ignore_index=True).set_index(['Date','TagId'])

def filter_data(df, mv, ts, xb, yb, progress_callback=None):
    m = (df['X']>=xb[0])&(df['X']<=xb[1])&(df['Y']>=yb[0])&(df['Y']<=yb[1])
    df.loc[~m, ['X','Y']] = np.nan
    dfv = calc_vel(df, ts, progress_callback)
    df['Velocity'] = dfv['Velocity']
    df.loc[(df['Velocity']>mv).fillna(False), ['X','Y','Velocity']] = np.nan
    return df

def calc_dir(df, sw=20, ts=100, stat_thresh=0.1, progress_callback=None):
    dt = ts/1000
    res = []
    groups = list(df.groupby(['Date','TagId']))
    for idx, ((d,t), g) in enumerate(groups):
        if progress_callback:
            progress_callback(0.6 + idx / len(groups) * 0.1, f'Directionality {idx+1}/{len(groups)}')
        n, times = len(g), g['Time'].values
        if n < 3:
            sub = pd.DataFrame({'Time':times, 'Directionality':np.full(n,np.nan)})
        else:
            x, y = g['X'].values.astype(float), g['Y'].values.astype(float)
            vx, vy = np.gradient(x,dt), np.gradient(y,dt)
            vx[np.isnan(x)], vy[np.isnan(y)] = np.nan, np.nan
            d_raw = np.arctan2(vy, vx)
            cplx = np.exp(1j*d_raw)
            sm = pd.Series(cplx).rolling(sw, center=True, min_periods=1).mean().values
            d_sm = np.angle(sm)
            spd = np.sqrt(vx**2+vy**2)
            d_sm[spd<stat_thresh] = np.nan
            sub = pd.DataFrame({'Time':times, 'Directionality':d_sm.astype(float)})
        sub['Date'], sub['TagId'] = d, t
        res.append(sub)
    dfD = pd.concat(res, ignore_index=True).set_index(['Date','TagId'])
    df['Directionality'] = dfD['Directionality']
    return df

def process_all(raw, ts, mv, se, sw, xb, yb, progress_callback=None):
    r = resample_data(raw, ts, progress_callback)
    df, tags, dates = reindex_data(r, ts)
    df = fill_smooth(df, ts, sw, se, progress_callback)
    df = filter_data(df, mv, ts, xb, yb, progress_callback)
    return df, tags, dates

print("✅ Processing functions loaded!")

✅ Processing functions loaded!


In [3]:
# =============================================================================
# PAIRWISE ANALYSIS FUNCTIONS
# =============================================================================

def calc_pw(dfd, tags, cd, ca):
    ca_r = np.radians(ca)
    dfd = dfd.reset_index()
    times = sorted(dfd['Time'].unique())
    nt, ntg = len(times), len(tags)
    piv = dfd.pivot(index='Time', columns='TagId', values=['X','Y','Directionality'])
    X, Y, D = [np.full((nt,ntg),np.nan) for _ in range(3)]
    for j,tg in enumerate(tags):
        if ('X',tg) in piv.columns:
            X[:,j], Y[:,j], D[:,j] = piv[('X',tg)].values, piv[('Y',tg)].values, piv[('Directionality',tg)].values
    dX, dY = X[:,:,None]-X[:,None,:], Y[:,:,None]-Y[:,None,:]
    Dist = np.sqrt(dX**2+dY**2)
    prox = Dist < cd
    ai2j, aj2i = np.arctan2(dY,dX), np.arctan2(-dY,-dX)
    Di, Dj = D[:,:,None], D[:,None,:]
    def cdiff(a,b): d=a-b; return np.abs(np.arctan2(np.sin(d),np.cos(d)))
    ri, rj = cdiff(Di,ai2j), cdiff(Dj,aj2i)
    view = (ri<ca_r)&(rj<ca_r)
    bd = ~np.isnan(Di)&~np.isnan(Dj)
    view = view & bd
    bp = ~np.isnan(X[:,:,None])&~np.isnan(X[:,None,:])
    return {'times':times,'tags':tags,'Dist':Dist,'prox':prox,'view':view,'bd':bd,'bp':bp,'ri':ri,'rj':rj}

def find_eps(arr, ms):
    arr = np.where(np.isnan(np.asarray(arr,float)),0,arr).astype(int)
    p = np.concatenate([[0],arr,[0]])
    d = np.diff(p)
    return [(s,e) for s,e in zip(np.where(d==1)[0],np.where(d==-1)[0]) if e-s>=ms]

def extract_eps(pw, tags, cdur, ts):
    ms = int(cdur*1000/ts)
    Dist, prox, view, bd, bp = pw['Dist'], pw['prox'], pw['view'], pw['bd'], pw['bp']
    n = len(tags)
    res = {'all_data':{'p':[],'i':[]},'moving_only':{'p':[],'i':[]}}
    for i in range(n):
        for j in range(i+1,n):
            ta, tb, d = tags[i], tags[j], Dist[:,i,j]
            def add(arr,m,typ):
                for s,e in find_eps(arr,ms):
                    seg=d[s:e]
                    res[m][typ].append({'TagId_A':ta,'TagId_B':tb,'start':s,'end':e,
                        'duration_sec':(e-s)*ts/1000,'mean_dist':np.nanmean(seg),
                        'min_dist':np.nanmin(seg) if np.any(~np.isnan(seg)) else np.nan})
            pa = prox[:,i,j]&bp[:,i,j]; add(pa,'all_data','p'); add(pa&view[:,i,j],'all_data','i')
            pm = prox[:,i,j]&bd[:,i,j]; add(pm,'moving_only','p'); add(pm&view[:,i,j],'moving_only','i')
    return res

def calc_dm(pw, tags):
    Dist, bp, bd = pw['Dist'], pw['bp'], pw['bd']
    n = len(tags)
    res = {'all_data':[],'moving_only':[]}
    for i in range(n):
        for j in range(i+1,n):
            d = Dist[:,i,j]
            for m,mk in [('all_data',bp[:,i,j]),('moving_only',bd[:,i,j])]:
                v = d[mk&~np.isnan(d)]
                if len(v)>0:
                    res[m].append({'TagId_A':tags[i],'TagId_B':tags[j],'mean':np.mean(v),'median':np.median(v),'min':np.min(v),'std':np.std(v),'n':len(v)})
    return res

def run_analysis(df, cd, ca, cdur, ts, progress_callback=None):
    ar = {'all_data':{'p':[],'i':[],'d':[]},'moving_only':{'p':[],'i':[],'d':[]}}
    all_pw = []
    dates = df.index.get_level_values('Date').unique()
    for idx, date in enumerate(dates):
        if progress_callback:
            progress_callback(0.7 + idx / len(dates) * 0.3, f'Analyzing date {idx+1}/{len(dates)}')
        dfd = df[df.index.get_level_values('Date')==date].copy()
        tags = list(dfd.index.get_level_values('TagId').unique())
        if len(tags)<2: continue
        pw = calc_pw(dfd, tags, cd, ca)
        all_pw.append(pw)
        eps = extract_eps(pw, tags, cdur, ts)
        for m in ['all_data','moving_only']:
            for e in eps[m]['p']: e['Date']=date
            for e in eps[m]['i']: e['Date']=date
            ar[m]['p'].extend(eps[m]['p']); ar[m]['i'].extend(eps[m]['i'])
        dm = calc_dm(pw, tags)
        for m in ['all_data','moving_only']:
            for x in dm[m]: x['Date']=date
            ar[m]['d'].extend(dm[m])
    out = {'_pw':all_pw}
    for m in ['all_data','moving_only']:
        out[m] = {'prox':pd.DataFrame(ar[m]['p']),'int':pd.DataFrame(ar[m]['i']),'dist':pd.DataFrame(ar[m]['d'])}
    return out

def comp_met(d):
    p, i, ds = d['prox'], d['int'], d['dist']
    return {
        'mean_dist': ds['mean'].mean() if len(ds)>0 else 0,
        'prox_n': len(p),
        'prox_time': p['duration_sec'].sum() if len(p)>0 else 0,
        'prox_dur': p['duration_sec'].mean() if len(p)>0 else 0,
        'int_n': len(i),
        'int_time': i['duration_sec'].sum() if len(i)>0 else 0,
        'int_dur': i['duration_sec'].mean() if len(i)>0 else 0,
        'ppl': len(set(i['TagId_A'].tolist()+i['TagId_B'].tolist())) if len(i)>0 else 0
    }

print("✅ Analysis functions loaded!")

✅ Analysis functions loaded!


In [4]:
# =============================================================================
# UI COMPONENTS
# =============================================================================

# Header
display(HTML('''
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; margin-bottom: 20px;">
    <h1 style="color: white; margin: 0;">📊 Social Interaction Analysis</h1>
    <p style="color: rgba(255,255,255,0.9); margin: 5px 0 0 0;">
        Enter folder path • Configure parameters • Analyze interactions
    </p>
</div>
'''))

# Data folder input (instead of file upload)
display(HTML('<h3>📁 1. Data Location</h3>'))
display(HTML('<p>Enter the full path to your folder containing CSV files:</p>'))

folder_input = widgets.Text(
    value=r'C:\Users\Julius de gebruiker\Masterstage data analysis\Datatestklassen',
    description='Folder:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='700px')
)

load_button = widgets.Button(
    description='📂 Load Data',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

load_output = widgets.Output()

display(folder_input)
display(load_button)
display(load_output)

def on_load_click(b):
    with load_output:
        clear_output()
        folder = folder_input.value
        path = Path(folder)
        
        if not path.exists():
            print(f"❌ Folder not found: {folder}")
            return
        
        files = list(path.glob('*.csv'))
        if not files:
            print(f"❌ No CSV files found in: {folder}")
            return
        
        print(f"📂 Found {len(files)} CSV files")
        print("Loading...")
        
        try:
            dfs = []
            for f in files:
                try:
                    df = pd.read_csv(f, sep=';', decimal='.')
                except:
                    df = pd.read_csv(f)
                dfs.append(df)
                print(f"  ✓ {f.name}")
            
            state['raw_data'] = pd.concat(dfs, ignore_index=True)
            n_rows = len(state['raw_data'])
            n_tags = state['raw_data']['TagId'].nunique()
            
            print(f"\n✅ Loaded successfully!")
            print(f"   Total rows: {n_rows:,}")
            print(f"   Unique tags: {n_tags}")
            print(f"   Columns: {list(state['raw_data'].columns)}")
            print(f"\n📊 Data Preview:")
            display(state['raw_data'].head())
            
        except Exception as e:
            print(f"❌ Error loading data: {e}")

load_button.on_click(on_load_click)

Text(value='C:\\Users\\Julius de gebruiker\\Masterstage data analysis\\Datatestklassen', description='Folder:'…

Button(button_style='primary', description='📂 Load Data', layout=Layout(width='150px'), style=ButtonStyle())

Output()

In [13]:
# =============================================================================
# PARAMETERS
# =============================================================================

display(HTML('<h3>⚙️ 2. Configure Parameters</h3>'))

# Processing parameters
display(HTML('<b>Processing Settings</b>'))

timestep_slider = widgets.IntSlider(
    value=100, min=50, max=500, step=50,
    description='Timestep (ms):', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

max_velocity_slider = widgets.FloatSlider(
    value=6.5, min=1.0, max=15.0, step=0.5,
    description='Max velocity (m/s):', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

smoothing_slider = widgets.IntSlider(
    value=20, min=5, max=50, step=5,
    description='Smoothing window:', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

stationary_slider = widgets.FloatSlider(
    value=0.1, min=0.05, max=0.5, step=0.05,
    description='Stationary (m/s):', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

display(timestep_slider)
display(max_velocity_slider)
display(smoothing_slider)
display(stationary_slider)

# Interaction thresholds
display(HTML('<br><b>Interaction Thresholds</b>'))

distance_slider = widgets.FloatSlider(
    value=1.2, min=0.5, max=3.0, step=0.1,
    description='Proximity (m):', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

angle_slider = widgets.IntSlider(
    value=30, min=15, max=180, step=15,
    description='Facing angle (°):', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

duration_slider = widgets.FloatSlider(
    value=5.0, min=1.0, max=30.0, step=1.0,
    description='Min duration (s):', style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

display(distance_slider)
display(angle_slider)
display(duration_slider)

IntSlider(value=100, description='Timestep (ms):', layout=Layout(width='450px'), max=500, min=50, step=50, sty…

FloatSlider(value=6.5, description='Max velocity (m/s):', layout=Layout(width='450px'), max=15.0, min=1.0, ste…

IntSlider(value=20, description='Smoothing window:', layout=Layout(width='450px'), max=50, min=5, step=5, styl…

FloatSlider(value=0.1, description='Stationary (m/s):', layout=Layout(width='450px'), max=0.5, min=0.05, step=…

FloatSlider(value=1.2, description='Proximity (m):', layout=Layout(width='450px'), max=3.0, min=0.5, style=Sli…

IntSlider(value=30, description='Facing angle (°):', layout=Layout(width='450px'), max=180, min=15, step=15, s…

FloatSlider(value=5.0, description='Min duration (s):', layout=Layout(width='450px'), max=30.0, min=1.0, step=…

In [14]:
# =============================================================================
# PROCESS BUTTON
# =============================================================================

display(HTML('<h3>🚀 3. Process & Analyze</h3>'))

process_button = widgets.Button(
    description='🚀 Process Data',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

progress_bar = widgets.FloatProgress(
    value=0, min=0, max=1.0,
    description='Progress:',
    bar_style='info',
    layout=widgets.Layout(width='500px')
)

status_label = widgets.HTML(value='<i>Load data first, then click Process</i>')
process_output = widgets.Output()

display(process_button)
display(progress_bar)
display(status_label)
display(process_output)

def update_progress(value, message):
    progress_bar.value = value
    status_label.value = f'<b>{message}</b>'

def on_process_click(b):
    if state['raw_data'] is None:
        status_label.value = '<b style="color:red">❌ Please load data first!</b>'
        return
    
    process_button.disabled = True
    progress_bar.value = 0
    
    with process_output:
        clear_output()
        
        try:
            # Parameters
            ts = timestep_slider.value
            mv = max_velocity_slider.value
            sw = smoothing_slider.value
            se = 0.3  # Standard error
            cd = distance_slider.value
            ca = angle_slider.value
            cdur = duration_slider.value
            stat_thresh = stationary_slider.value
            
            # Spatial bounds
            MARGIN = 1.0
            raw = state['raw_data']
            X_B = (raw['X'].quantile(0.001)-MARGIN, raw['X'].quantile(0.999)+MARGIN)
            Y_B = (raw['Y'].quantile(0.001)-MARGIN, raw['Y'].quantile(0.999)+MARGIN)
            
            print(f"📐 Spatial bounds: X=[{X_B[0]:.1f}, {X_B[1]:.1f}], Y=[{Y_B[0]:.1f}, {Y_B[1]:.1f}]")
            
            # Process
            update_progress(0.05, '⚙️ Processing data...')
            df, tags_dict, dates_list = process_all([raw], ts, mv, se, sw, X_B, Y_B, update_progress)
            print(f"✓ Processed: {len(df):,} rows, {len(tags_dict)} tags, {len(dates_list)} dates")
            
            update_progress(0.6, '🧭 Calculating directionality...')
            df = calc_dir(df, sw, ts, stat_thresh, update_progress)
            print(f"✓ Directionality calculated")
            
            update_progress(0.7, '👥 Running pairwise analysis...')
            results = run_analysis(df, cd, ca, cdur, ts, update_progress)
            print(f"✓ Analysis complete")
            
            # Compute metrics
            metrics = {
                'all_data': comp_met(results['all_data']),
                'moving_only': comp_met(results['moving_only'])
            }
            
            # Store in state
            state['processed_data'] = df
            state['results'] = results
            state['metrics'] = metrics
            state['tags_dict'] = tags_dict
            state['dates_list'] = dates_list
            
            update_progress(1.0, '✅ Processing complete!')
            print("\n" + "="*50)
            print("✅ PROCESSING COMPLETE!")
            print("="*50)
            print("\nRun the next cells to see results and visualizations.")
            
        except Exception as e:
            status_label.value = f'<b style="color:red">❌ Error: {str(e)}</b>'
            import traceback
            print(traceback.format_exc())
    
    process_button.disabled = False

process_button.on_click(on_process_click)

Button(button_style='success', description='🚀 Process Data', layout=Layout(height='40px', width='200px'), styl…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(width='500px'), max=1.0)

HTML(value='<i>Load data first, then click Process</i>')

Output()

In [15]:
# =============================================================================
# RESULTS DISPLAY
# =============================================================================

display(HTML('<h3>📊 4. Results</h3>'))

results_output = widgets.Output()
display(results_output)

def show_results():
    with results_output:
        clear_output()
        
        if state['metrics'] is None:
            print("⚠️ No results yet. Run processing first.")
            return
        
        metrics = state['metrics']
        
        # Summary cards HTML
        html = '<div style="display: flex; flex-wrap: wrap; gap: 10px; margin-bottom: 20px;">'
        
        cards = [
            ('Mean Distance', f"{metrics['all_data']['mean_dist']:.2f} m", '#667eea'),
            ('Proximity Episodes', f"{metrics['all_data']['prox_n']}", '#f093fb'),
            ('Time in Proximity', f"{metrics['all_data']['prox_time']/60:.1f} min", '#4facfe'),
            ('Interaction Episodes', f"{metrics['all_data']['int_n']}", '#43e97b'),
            ('Time in Interaction', f"{metrics['all_data']['int_time']/60:.1f} min", '#fa709a'),
            ('People Interacting', f"{metrics['all_data']['ppl']}", '#f5576c'),
        ]
        
        for label, value, color in cards:
            html += f'''
            <div style="background: {color}; border-radius: 10px; padding: 15px; 
                        color: white; text-align: center; min-width: 140px;">
                <div style="font-size: 22px; font-weight: bold;">{value}</div>
                <div style="font-size: 11px; opacity: 0.9;">{label}</div>
            </div>
            '''
        html += '</div>'
        display(HTML(html))
        
        # Comparison table
        print("\n📈 Comparison: All Data vs Moving Only")
        print("="*60)
        print(f"{'Metric':<30}{'All Data':>15}{'Moving Only':>15}")
        print("-"*60)
        
        rows = [
            ('Mean Distance (m)', 'mean_dist', '.2f'),
            ('Proximity Episodes', 'prox_n', 'd'),
            ('Proximity Time (s)', 'prox_time', '.1f'),
            ('Interaction Episodes', 'int_n', 'd'),
            ('Interaction Time (s)', 'int_time', '.1f'),
            ('People in Interactions', 'ppl', 'd'),
        ]
        
        for label, key, fmt in rows:
            av = metrics['all_data'][key]
            mv = metrics['moving_only'][key]
            print(f"{label:<30}{av:>15{fmt}}{mv:>15{fmt}}")
        print("-"*60)

# Auto-show results if available
show_results()

Output()

In [16]:
# =============================================================================
# VISUALIZATIONS
# =============================================================================

display(HTML('<h3>📈 5. Visualizations</h3>'))

viz_output = widgets.Output()
display(viz_output)

def show_visualizations():
    with viz_output:
        clear_output()
        
        if state['processed_data'] is None:
            print("⚠️ No data yet. Run processing first.")
            return
        
        df = state['processed_data']
        results = state['results']
        metrics = state['metrics']
        tags_dict = state['tags_dict']
        
        # Figure 1: Spatial overview
        print("📍 Spatial Overview")
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        df_plot = df.reset_index()
        
        ax1 = axes[0]
        x_data = df_plot['X'].dropna()
        y_data = df_plot['Y'].dropna()
        if len(x_data) > 0:
            hb = ax1.hexbin(x_data, y_data, gridsize=30, cmap='YlOrRd', mincnt=1)
            plt.colorbar(hb, ax=ax1, label='Count')
        ax1.set_xlabel('X (m)'); ax1.set_ylabel('Y (m)')
        ax1.set_title('Position Density', fontweight='bold')
        ax1.set_aspect('equal')
        
        ax2 = axes[1]
        sample_tags = list(tags_dict.values())[:5]
        for i, tag in enumerate(sample_tags):
            td = df_plot[df_plot['TagId']==tag].iloc[::10]
            ax2.plot(td['X'], td['Y'], alpha=0.5, linewidth=0.5, label=str(tag)[:8])
        ax2.set_xlabel('X (m)'); ax2.set_ylabel('Y (m)')
        ax2.set_title('Sample Trajectories', fontweight='bold')
        ax2.set_aspect('equal')
        ax2.legend(fontsize=8)
        plt.tight_layout()
        plt.show()
        
        # Figure 2: Metrics comparison
        print("\n📊 Metrics Comparison")
        fig, axes = plt.subplots(1, 3, figsize=(14, 4))
        cats = ['Proximity', 'Interaction']
        x = np.arange(len(cats))
        w = 0.35
        
        ax1 = axes[0]
        v1 = [metrics['all_data']['prox_n'], metrics['all_data']['int_n']]
        v2 = [metrics['moving_only']['prox_n'], metrics['moving_only']['int_n']]
        ax1.bar(x-w/2, v1, w, label='All Data', color='steelblue')
        ax1.bar(x+w/2, v2, w, label='Moving', color='coral')
        ax1.set_xticks(x); ax1.set_xticklabels(cats)
        ax1.set_ylabel('Episodes'); ax1.set_title('Episode Counts', fontweight='bold')
        ax1.legend()
        
        ax2 = axes[1]
        v1 = [metrics['all_data']['prox_time']/60, metrics['all_data']['int_time']/60]
        v2 = [metrics['moving_only']['prox_time']/60, metrics['moving_only']['int_time']/60]
        ax2.bar(x-w/2, v1, w, label='All Data', color='steelblue')
        ax2.bar(x+w/2, v2, w, label='Moving', color='coral')
        ax2.set_xticks(x); ax2.set_xticklabels(cats)
        ax2.set_ylabel('Time (min)'); ax2.set_title('Total Time', fontweight='bold')
        ax2.legend()
        
        ax3 = axes[2]
        v1 = [metrics['all_data']['prox_dur'], metrics['all_data']['int_dur']]
        v2 = [metrics['moving_only']['prox_dur'], metrics['moving_only']['int_dur']]
        ax3.bar(x-w/2, v1, w, label='All Data', color='steelblue')
        ax3.bar(x+w/2, v2, w, label='Moving', color='coral')
        ax3.set_xticks(x); ax3.set_xticklabels(cats)
        ax3.set_ylabel('Duration (s)'); ax3.set_title('Mean Duration', fontweight='bold')
        ax3.legend()
        plt.tight_layout()
        plt.show()
        
        # Figure 3: Distributions
        if len(results['all_data']['dist']) > 0 or len(results['all_data']['prox']) > 0:
            print("\n📉 Distributions")
            fig, axes = plt.subplots(1, 2, figsize=(12, 4))
            
            if len(results['all_data']['dist']) > 0:
                ax1 = axes[0]
                d = results['all_data']['dist']['mean']
                ax1.hist(d, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
                ax1.axvline(distance_slider.value, color='red', linestyle='--', lw=2, 
                           label=f'Threshold ({distance_slider.value}m)')
                ax1.axvline(d.mean(), color='orange', linestyle='-', lw=2, 
                           label=f'Mean ({d.mean():.2f}m)')
                ax1.set_xlabel('Mean Distance (m)'); ax1.set_ylabel('Pairs')
                ax1.set_title('Distance Distribution', fontweight='bold')
                ax1.legend()
            
            if len(results['all_data']['prox']) > 0:
                ax2 = axes[1]
                dur = results['all_data']['prox']['duration_sec']
                ax2.hist(dur, bins=50, edgecolor='black', alpha=0.7, color='coral')
                ax2.axvline(dur.mean(), color='darkred', lw=2, label=f'Mean ({dur.mean():.1f}s)')
                ax2.set_xlabel('Duration (s)'); ax2.set_ylabel('Episodes')
                ax2.set_title('Proximity Episode Duration', fontweight='bold')
                if len(dur) > 0:
                    ax2.set_xlim(0, np.percentile(dur, 95))
                ax2.legend()
            plt.tight_layout()
            plt.show()

# Auto-show if data available
show_visualizations()

Output()

In [9]:
# =============================================================================
# SAVE RESULTS
# =============================================================================

display(HTML('<h3>💾 6. Save Results</h3>'))

save_folder_input = widgets.Text(
    value=r'C:\Users\Julius de gebruiker\Downloads',
    description='Save to:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='700px')
)

save_button = widgets.Button(
    description='💾 Save All Results',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

save_output = widgets.Output()

display(save_folder_input)
display(save_button)
display(save_output)

def on_save_click(b):
    with save_output:
        clear_output()
        
        if state['results'] is None:
            print("❌ No results to save. Run processing first.")
            return
        
        save_path = Path(save_folder_input.value)
        if not save_path.exists():
            print(f"❌ Folder not found: {save_path}")
            return
        
        results = state['results']
        metrics = state['metrics']
        df = state['processed_data']
        
        print("💾 Saving results...")
        
        # Save processed data
        df.reset_index().to_csv(save_path / 'processed_data.csv', index=False)
        print(f"  ✓ processed_data.csv")
        
        # Save episode data
        for mode, suffix in [('all_data','_all'), ('moving_only','_moving')]:
            for name, key in [('proximity_episodes','prox'), ('interaction_episodes','int'), ('distance_by_dyad','dist')]:
                df_data = results[mode][key]
                if len(df_data) > 0:
                    filename = f'{name}{suffix}.csv'
                    df_data.to_csv(save_path / filename, index=False)
                    print(f"  ✓ {filename} ({len(df_data)} rows)")
        
        # Save metrics summary
        metrics_df = pd.DataFrame([{'mode':k, **v} for k,v in metrics.items()])
        metrics_df.to_csv(save_path / 'metrics_summary.csv', index=False)
        print(f"  ✓ metrics_summary.csv")
        
        print(f"\n✅ All files saved to: {save_path}")

save_button.on_click(on_save_click)

Text(value='C:\\Users\\Julius de gebruiker\\Downloads', description='Save to:', layout=Layout(width='700px'), …

Button(button_style='success', description='💾 Save All Results', layout=Layout(width='200px'), style=ButtonSty…

Output()

In [10]:
# =============================================================================
# REFRESH BUTTONS
# =============================================================================

display(HTML('<hr><h3>🔄 Refresh Views</h3>'))
display(HTML('<p>After processing, click these to update the displays:</p>'))

refresh_results_btn = widgets.Button(description='🔄 Refresh Results', button_style='info')
refresh_viz_btn = widgets.Button(description='🔄 Refresh Visualizations', button_style='info')

def on_refresh_results(b):
    show_results()

def on_refresh_viz(b):
    show_visualizations()

refresh_results_btn.on_click(on_refresh_results)
refresh_viz_btn.on_click(on_refresh_viz)

display(widgets.HBox([refresh_results_btn, refresh_viz_btn]))